# 4 ニューラルネットワークの学習

本章のテーマは、ニューラルネットワークの学習です。ここでいう「学習」とは、訓練データから最適な重みパラメータの値を自動で獲得することを指します。本章では、ニューラルネットワークが学習を行えるようにするために、損失関数という「指標」を導入します。この損失関数を基準として、その値が最も小さくなる重みパラメータを探し出すということが学習の目的です。本章では、できるだけ小さな損失関数の値を探し出すための手法として、勾配法と呼ばれる、関数の傾きを使った手法を説明します。

## 4.1 データから学習する

### 4.1.1 データ駆動

ゼロから「５」を認識するアルゴリズムを"ひねり出す"代わりに、データを有効に活用して解決したいと考えます。

ニューラルネットワークは、画像を"そのまま"学習思案す。特徴量と機械学習によるアプローチの例では人が特徴量を設計しましたが、ニューラルネットワークは、画像に含まれる重要な特徴量までも「機械」が学習するのです。

### 4.1.2 訓練データとテストデータ

機械学習の問題では、**訓練データ**と**テストデータ**の２つのデータに分けて、学習や実験などを行うのが一般的です。その場合、まずは訓練データだけを使って学習を行い、最適なパラメータを探索します。そして　、テストデータを使って、その訓練したモデルの実力を評価するのです。

**汎化能力**とは、まだ見ぬデータ（訓練データに含まれないデータ）に対しての能力であり、この汎化能力を獲得することこそが機械学習の最終的な目標です。

あるデータセットだけに過度に対応した状態を**過学習**（overwriting）と言います。過学習を避けることは、機械学習の重要な課題でもあります。

## 4.2 損失関数

ニューラルネットワークは「ひとつの指標」を手掛かりに最適なパラメータを探索します。なお、ニューラルネットワークの学習で用いられる指標は、**損失関数**（loss function）と呼ばれます。この損失関数は、任意の関数を用いることができますが、一般的には、２乗和誤差や交差エントロピー誤差などが用いられます。

### 4.2.1 ２乗和誤差

$$
E = \frac{1}{2}\sum_{k}(y_k - t_k)^2
\tag{4.1}
$$

ここで、$y_k$はニューラルネットワークの出力、$t_k$は教師データを表し、$k$はデータの次元数を表します。

In [ ]:
import numpy as np

def sum_squared_error(y, t):
    return 0.5 * np.sum((y-t)**2)

# 「２」を正解とする
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
# 「２」の確率が最も高い場合
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
sum_squared_error(np.array(y), np.array(t))

0.09750000000000003

In [ ]:
# 「２」の確率が最も高い場合
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.1, 0.6, 0.0]
sum_squared_error(np.array(y), np.array(t))

0.6025

ひとつ目の例のほうが、出力結果が教師データにより適合していることを２乗和誤差は示しているのです。

### 4.2.2 交差エントロピー誤差

$$
E = -\sum_{k}t_k\log{y_k}
\tag{4.2}
$$

In [ ]:
def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

np.logの計算時に、微小な値であるdeltaを足して計算しています。これは、np.log(0)のような計算が発生した場合にもエラーとならないようにするためです。

In [ ]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [ ]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.1, 0.6, 0.0]
cross_entropy_error(np.array(y), np.array(t))

2.302584092994546

これらの結果から、これまでの議論と一致していることがわかります。

### 4.2.3 ミニバッチ学習

機械学習の問題は、訓練データを使って学習を行います。訓練データを使って学習するとは、正確に言うと、訓練データに対する損失関数を求め、その値をできるだけ小さくするようなパラメータを探し出す、ということです。そのため、損失関数は、すべての訓練データを対象として求める必要があります。つまり、訓練データが100個あれば、その100個の和を損失関数の指標とするのです。

先ほど説明した損失関数の例は、ひとつのデータの損失関数を考えていました。そこで、訓練データすべての損失関数の和を求めたいとすると、たとえば、交差エントロピーの場合、次の式（4.3）のように書くことができます。

$$
E = -\frac{1}{N}\sum_{n}\sum_{k}t_{nk}\log{y_{nk}}
\tag{4.3}
$$

ここで、データがN個あるとして、$t_{nk}$は$n$番目のデータの$k$番目の値を意味します。（$y_{nk}$はニューラルネットワークの出力、$t_{nk}$は教師データです）。

すべてのデータを対象とした損失関数を計算するのは、現実的ではありません。そこで、データの中から一部を選び出し、その一部のデータを全体の「近似」として利用します。ニューラルネットワークの学習においても、訓練データからある枚数だけを選び出し-これをミニバッチ（小さな塊）という-、そのミニバッチごとに学習を行います。このような学習手法を**ミニバッチ学習**と言います。

In [ ]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) =\
load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape)
print(t_train.shape)

(60000, 784)
(60000, 10)


それでは、この訓練データの中からランダムに10枚だけ抜き出すには、どうすればよいでしょうか？

In [ ]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
print(batch_mask)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]
# print(x_batch)
# print(t_batch)

[52540 34922 22312 46527   602   192 21092 53128 10407 57973]


np.random.choice()を使えば、指定された数字の中からランダムに好きな数だけ取り出すことができます。

In [ ]:
np.random.choice(60000,10)

array([21741, 47663, 36656, 38215, 12500, 22795, 37745, 55520, 10031,
       13851])

後は、このランダムに選ばれたインデックスを指定して、ミニバッチを取り出すだけです。このミニバッチを使って、損失関数を計算します。

In [ ]:
print(np.log(np.array([1.0,2.0,3.0])+3.0))

[1.38629436 1.60943791 1.79175947]


In [ ]:
print(np.log(4))

1.3862943611198906


### 4.2.4 なぜ損失関数を設定するのか？

ニューラルネットワークの学習の際に、認識制度を"指標"にしてはいけない。その理由は、認識制度を指標にすると、パラメータの微分がほとんどの場所で０になってしまうからだ。

## 4.3 数値微分

aiueo

ffadfadfdsafdsfd